In [1]:
# f_X_train['review_removed_stopwords_length'] = f_X_train['review_removed_stopwords'].apply(lambda x: len(x.split()))

In [2]:
# max(f_X_train['review_length'])

In [3]:
# max(f_X_train['review_removed_stopwords_length'])

In [4]:
# get_plot(f_X_train)
# get_plot(f_X_train,'review_removed_stopwords_length')

In [5]:
# df_o = pd.DataFrame()
# df_o[['polarity', 'review']] = x_train[['polarity', 'review']].copy()
# df_n = pd.DataFrame()
# df_n[['polarity', 'review']] = f_X_train[['polarity', 'review_removed_stopwords']].copy()

In [6]:
# # 메모리 사용량 계산
# memory_usage = df_o.memory_usage(deep=True).sum()

# # 메모리 사용량을 MB 단위로 변환
# memory_usage_MB = memory_usage / (1024 ** 2)

# # 메모리 사용량 계산
# memory_usage_f = df_n.memory_usage(deep=True).sum()

# # 메모리 사용량을 MB 단위로 변환
# memory_usage_MB_f = memory_usage_f / (1024 ** 2)

# # 메모리 사용량 출력
# print(f"Memory usage of df_train: {memory_usage_MB:.2f}MB / Filtered: {memory_usage_MB_f:.2f}MB")

In [7]:
# df_f_o = pd.DataFrame()
# df_f_o = filter_top_70_percent(df_o)
# df_f_n = pd.DataFrame()
# df_f_n = filter_top_70_percent(df_n)

In [8]:
# # 메모리 사용량 계산
# memory_usage = df_f_o.memory_usage(deep=True).sum()

# # 메모리 사용량을 MB 단위로 변환
# memory_usage_MB = memory_usage / (1024 ** 2)

# # 메모리 사용량 계산
# memory_usage_f = df_f_n.memory_usage(deep=True).sum()

# # 메모리 사용량을 MB 단위로 변환
# memory_usage_MB_f = memory_usage_f / (1024 ** 2)

# # 메모리 사용량 출력
# print(f"Memory usage of df_train: {memory_usage_MB:.2f}MB / Filtered: {memory_usage_MB_f:.2f}MB")

In [9]:
# df_f_n

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os

import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
import numpy as np
import multiprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import dask.dataframe as dd

class TextPreprocessor:
    # Example usage:
    _stop_words = set(stopwords.words('english'))
    def __init__(self, max_features=5000, max_length = 100, input_dir = 'InputData', data_set_header = ['polarity', 'title', 'review']):
        """
        Constructor to initialize paths, column headers, and TF-IDF settings.
        
        :param max_features: [Maximum number of words in the vocabulary] or [Maximum number of words to use for TF-IDF vectorization]
        :param max_length: Maximum length of the input sequences
        """
        self.PATH_TRAIN_DATA = os.path.join(input_dir, 'train.csv')
        self.PATH_TEST_DATA = os.path.join(input_dir, 'test.csv')
        self.PATH_TRAIN_CLEANED_DATA=os.path.join(input_dir, 'train_cleaned.csv')
        self.PATH_TEST_CLEANED_DATA=os.path.join(input_dir, 'test_cleaned.csv')
        self.DATA_SET_HEADER = data_set_header
        self.max_features = max_features
        self.max_length = max_length

    def filter_by_length_of_sentence(self, X_train, percent=70):
        X_train['review_length'] = X_train['review'].apply(len)
        
        length_counts = X_train['review_length'].value_counts().reset_index()
        length_counts.columns = ['review_length', 'count']
        length_counts = length_counts.sort_values('review_length')
        
        length_counts['cumulative_sum'] = length_counts['count'].cumsum()
        length_counts['cumulative_percentage'] = 100 * length_counts['cumulative_sum'] / length_counts['count'].sum()
        
        threshold = length_counts[length_counts['cumulative_percentage'] >= percent].iloc[0]['review_length']
        self.max_length = threshold+1
        filtered_data = X_train[X_train['review_length'] <= threshold]
        
        return filtered_data
    
    def limit_length_of_sentence(self, X_train, max_length):
        """
        Limit the length of sentences in the 'review' column to the specified max_length.
        Any sentence exceeding the max_length will be truncated.

        Parameters:
        X_train: pd.DataFrame
            DataFrame containing the text data. It must have a 'review' column with text to be processed.
        max_length: int
            The maximum allowed length of a sentence. Sentences longer than this value will be truncated.

        Returns:
        pd.DataFrame
            A DataFrame with sentences truncated to the specified max_length.
        """

        # Truncate each review in 'review' column to max_length
        X_train['review_l'] = X_train['review'].apply(lambda x: ' '.join(x.split()[:max_length]) if len(x.split()) > max_length else x) # Use .copy() to avoid unintended effects on other columns
    
        
        return X_train


    def sampling_data(self,df, num_sample=10000):
        """
        Random sampling of {num_sample} reviews
        
        :param df: Input DataFrame
        :param num_sample: Number to sample
        :return: Sampled DataFrame
        """
        df = df.sample(n=num_sample, random_state=42)  # random_state ensures reproducibility

        return df

    def load_data(self, num_sample=0, test_ratio=0.2, los = 70):
        """
        Load training and test datasets.
        
        :return: Loaded DataFrames for train and test datasets
        """
        df_train = pd.read_csv(self.PATH_TRAIN_DATA, names=self.DATA_SET_HEADER)
        # df_train = self.filter_by_length_of_sentence(df_train)

        df_test = pd.read_csv(self.PATH_TEST_DATA, names=self.DATA_SET_HEADER)
        if num_sample == 0:
            pass
        else:
            df_train = self.sampling_data(df_train, num_sample=num_sample)
            df_test = self.sampling_data(df_test, num_sample=int(num_sample*test_ratio))

        df_train = df_train.dropna()
        df_test = df_test.dropna()

        return df_train, df_test

    def parallel_load_data(self, num_sample=0, test_ratio=0.2, los = 70):
        """
        Load training and test datasets.
        
        :return: Loaded DataFrames for train and test datasets
        """
        df_train = dd.read_csv(self.PATH_TRAIN_DATA, names=self.DATA_SET_HEADER)
        df_test = dd.read_csv(self.PATH_TEST_DATA, names=self.DATA_SET_HEADER)

        df_train = df_train.compute()
        df_test = df_test.compute()

        # df_train = pd.read_csv(self.PATH_TRAIN_DATA, names=self.DATA_SET_HEADER)
        # # df_train = self.filter_by_length_of_sentence(df_train)

        # df_test = pd.read_csv(self.PATH_TEST_DATA, names=self.DATA_SET_HEADER)
        if num_sample == 0:
            pass
        else:
            df_train = self.sampling_data(df_train, num_sample=num_sample)
            df_test = self.sampling_data(df_test, num_sample=int(num_sample*test_ratio))

        df_train = df_train.dropna()
        df_test = df_test.dropna()

        return df_train, df_test
    
    def preprocess(self, df):
        """
        Preprocess the dataset by converting polarity to binary labels.
        
        :param df: Input DataFrame
        :return: Preprocessed DataFrame with binary labels
        """
        df['polarity'] = df['polarity'].map({1: 0, 2: 1})  # 1 for negative, 2 for positive
        return df

    def split_data(self, df):
        """
        Split the dataset into training and test sets.
        
        :param df: Input DataFrame
        :return: Training and test sets for X and y
        """
        X = df['review']  # Feature: review text
        y = df['polarity']  # Target: sentiment polarity

        return X, y

    def vectorize_text(self, X_train, X_test):
        """
        Perform TF-IDF vectorization on the training and test datasets.
        
        :param X_train: Training text data
        :param X_test: Test text data
        :return: TF-IDF transformed X_train and X_test
        """
        X_train_tfidf = self.vectorizer.fit_transform(X_train)
        X_test_tfidf = self.vectorizer.transform(X_test)
        return X_train_tfidf, X_test_tfidf
    
    def tokenization_and_padding(self, X_train, X_test):
        """
        Tokenization and Padding
        Pad sequences to ensure uniform input length

        :param X_train: Training text data
        :param X_test: Test text data
        :return: Padded Sequences
        """
        self.tokenizer = Tokenizer(num_words=self.max_features)
        self.tokenizer.fit_on_texts(X_train)

        X_train_seq = self.tokenizer.texts_to_sequences(X_train)
        X_test_seq = self.tokenizer.texts_to_sequences(X_test)

        X_train_pad = pad_sequences(X_train_seq, maxlen=self.max_length)
        X_test_pad = pad_sequences(X_test_seq, maxlen=self.max_length)
        return X_train_pad, X_test_pad
    
    def _expand_contractions(self, review):
        contractions = {
            "don't": "do not", "I'm": "I am", "you're": "you are", "it's": "it is",
            "he's": "he is", "she's": "she is", "we're": "we are", "they're": "they are",
            "I've": "I have", "you've": "you have", "we've": "we have", "they've": "they have",
            "isn't": "is not", "aren't": "are not", "wasn't": "was not", "weren't": "were not",
            "haven't": "have not", "hasn't": "has not", "hadn't": "had not",
            "won't": "will not", "wouldn't": "would not", "can't": "cannot", "couldn't": "could not",
            "shouldn't": "should not", "mightn't": "might not", "mustn't": "must not"
        }
        
        contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
        
        def replace(match):
            return contractions[match.group(0)]
    
        return contractions_re.sub(replace, review)

    def _remove_stopwords(self, review):
        review = self._expand_contractions(review)
        
        word_tokens = word_tokenize(review)
        
        filtered_review = [re.sub(r'\W+$', '', word.lower()) for word in word_tokens if word.lower() not in self._stop_words]
        
        filtered_review = ' '.join(filtered_review).strip()
        filtered_review = re.sub(' +', ' ', filtered_review) 
        
        return filtered_review

    def _process_reviews_chunk(self, reviews_chunk):
        return reviews_chunk.apply(self._remove_stopwords)
    
    def remove_stopwords(self, df, col_name='review'):
        num_cores = multiprocessing.cpu_count() 
        pool = multiprocessing.Pool(num_cores)   

        review_chunks = np.array_split(df[col_name], num_cores)
        processed_chunks = pool.map(self._process_reviews_chunk, review_chunks)

        df[col_name] = pd.concat(processed_chunks)
        
        pool.close()
        pool.join()

        return df
    
    def count_sentence_words_count(self, df, col_name='review'):
        df[f'{col_name}_length'] = df[col_name].apply(lambda x: len(x.split()))
        return df
    
    def get_plot(self, df,col_name='review_length'):
        review_length_counts = df[col_name].value_counts().sort_index()

        cumulative_percentage = np.cumsum(review_length_counts.values) / np.sum(review_length_counts.values) * 100

        length_70_percent = review_length_counts.index[np.argmax(cumulative_percentage >= 70)]
        length_80_percent = review_length_counts.index[np.argmax(cumulative_percentage >= 80)]

        plt.figure(figsize=(20, 6))
        plt.bar(review_length_counts.index, review_length_counts.values, color='skyblue')
        plt.axvline(x=length_70_percent, color='red', linestyle='--', label=f'70% at {length_70_percent} words')
        plt.axvline(x=length_80_percent, color='green', linestyle='--', label=f'80% at {length_80_percent} words')
        plt.xlabel('Review Length (Number of Words)', fontsize=12)
        plt.ylabel('Number of Reviews', fontsize=12)
        plt.title('Distribution of Review Lengths in f_X_train', fontsize=14)
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()

        print('Words count - 70%: ',length_70_percent)
        print('Words count - 80%: ',length_80_percent)

2024-10-11 18:28:28.536481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 18:28:28.880276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu:/usr/local/cuda-12.3/lib64:/usr/local/cuda-12.3/lib64:/usr/local/cuda-11.8/lib64
2024-10-11 18:28:28.880327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-g

In [2]:
processor = TextPreprocessor()
df_train, df_test = processor.parallel_load_data()

In [12]:
# processor = TextPreprocessor()
# df_train, df_test = processor.load_data()

In [3]:
df_train_f = processor.remove_stopwords(df_train.copy())
df_test_f = processor.remove_stopwords(df_test.copy())

/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [4]:
df_train_f = processor.filter_by_length_of_sentence(df_train_f)
df_test_f = processor.filter_by_length_of_sentence(df_test_f)

In [5]:
df_train_f = processor.preprocess(df_train_f)
df_test_f = processor.preprocess(df_test_f)

In [6]:
# Calulate Memory Usage
memory_usage = df_train.memory_usage(deep=True).sum()

# Convert Memory unit to MB
memory_usage_MB = memory_usage / (1024 ** 2)

# Calulate Memory Usage
memory_usage_f = df_train_f.memory_usage(deep=True).sum()

# Convert Memory unit to MB
memory_usage_MB_f = memory_usage_f / (1024 ** 2)

# Print Memory usage
print(f"Memory usage of df_train: {memory_usage_MB:.2f}MB / Filtered: {memory_usage_MB_f:.2f}MB")

Memory usage of df_train: 1927.45MB / Filtered: 798.20MB


In [7]:
df_train_f = processor.sampling_data(df_train_f, 1000)
df_test_f = processor.sampling_data(df_train_f, 200)

In [8]:
df_train_f.to_csv(processor.PATH_TRAIN_CLEANED_DATA, index=False)
df_test_f.to_csv(processor.PATH_TEST_CLEANED_DATA, index=False)

In [39]:
df_train_f = pd.read_csv(processor.PATH_TRAIN_CLEANED_DATA)
df_test_f = pd.read_csv(processor.PATH_TEST_CLEANED_DATA)

In [40]:
df_train

,polarity,title,review
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."
...,...,...,...
151083,1,Don't do it!!,The high chair looks great when it first comes...
151084,1,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
151085,1,"compact, but hard to clean","We have a small house, and really wanted two o..."
151086,1,what is it saying?,not sure what this book is supposed to be. It ...


In [41]:
df_train_f

,polarity,title,review,review_length
0,1,Stuning even for the non-gamer,sound track beautiful paints senery mind well ...,264
1,1,The best soundtrack ever to anything.,reading lot reviews saying best 'game soundtra...,290
2,1,"Remember, Pull Your Jaw Off The Floor After He...",played game know divine music every single son...,272
3,1,Glorious story,loved whisper wicked saints story amazing plea...,267
4,1,A FIVE STAR BOOK,finished reading whisper wicked saints fell lo...,301
...,...,...,...,...
2524031,1,AWESOME,absolutely amazing relieving neck pain invert ...,144
2524032,0,What A Slap In The Face To Masami Ueda,buy cd ever probably released test see refans ...,207
2524033,0,Too simplistic,mr harrison makes extremely valid arguments bo...,292
2524034,0,Don't do it!!,high chair looks great first comes box hill im...,207


In [42]:
# Split data
X_train, y_train= processor.split_data(df_train_f)
X_test, y_test = processor.split_data(df_test_f)

In [43]:
X_train_tfidf, X_test_tfidf = processor.vectorize_text(X_train, X_test)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [20]:
import sys
import os
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import joblib
# Importing necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


# Model building
# Using Logistic Regression for sentiment classification
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)

# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 86.98%
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.86      0.87    136933
           2       0.87      0.88      0.87    143561

    accuracy                           0.87    280494
   macro avg       0.87      0.87      0.87    280494
weighted avg       0.87      0.87      0.87    280494



In [21]:
param_dist= {
    'C':np.logspace(-3,1,10),
    'solver':['liblinear', 'saga','lbgfs'],
    'max_iter':[100, 200, 300],
    'penalty':['l1' ,'l2']
}

random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=100, cv=5, verbose=1, random_state=42)
random_search.fit(X_train_tfidf, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
140 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
140 fits failed with the following error:
Traceback (most recent call last):
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/woong/AI/Dissertation/.venv/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_params
    vali

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': array([1.00000000e-03, 2.78255940e-03, 7.74263683e-03, 2.15443469e-02,
       5.99484250e-02, 1.66810054e-01, 4.64158883e-01, 1.29154967e+00,
       3.59381366e+00, 1.00000000e+01]),
                                        'max_iter': [100, 200, 300],
                                        'penalty': ['l1', 'l2'],
                                        'solver': ['liblinear', 'saga',
                                                   'lbgfs']},
                   random_state=42, verbose=1)

In [23]:
# Making predictions
y_pred = random_search.predict(X_test_tfidf)

In [24]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Hypertuning Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Hypertuning Model Accuracy: 87.08%
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.86      0.87    136933
           2       0.87      0.88      0.87    143561

    accuracy                           0.87    280494
   macro avg       0.87      0.87      0.87    280494
weighted avg       0.87      0.87      0.87    280494



In [26]:
print("Best parameters found: ", random_search.best_params_)

Best parameters found:  {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 100, 'C': 0.46415888336127775}


In [27]:
model_f = LogisticRegression(**random_search.best_params_)

model_f.fit(X_train_tfidf, y_train)

LogisticRegression(C=0.46415888336127775, penalty='l1', solver='liblinear')

In [28]:
y_pred = model_f.predict(X_test_tfidf)

In [29]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Hypertuning Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Hypertuning Model Accuracy: 87.08%
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.86      0.87    136933
           2       0.87      0.88      0.87    143561

    accuracy                           0.87    280494
   macro avg       0.87      0.87      0.87    280494
weighted avg       0.87      0.87      0.87    280494



In [7]:
import joblib
import os

OUTPUT_DIR = "Output/data_full_random"

In [30]:
# 저장할 디렉토리 설정 (필요 시 생성)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Best parameters 저장
best_params_path = os.path.join(OUTPUT_DIR, 'best_params.pkl')
joblib.dump(random_search.best_params_, best_params_path)

print(f"Best parameters saved at: {best_params_path}")

# 최적의 파라미터를 사용하여 모델 생성
model_f = LogisticRegression(**random_search.best_params_)

# 모델 저장
model_path = os.path.join(OUTPUT_DIR, 'best_logistic_regression_model.pkl')
joblib.dump(model_f, model_path)

print(f"Best Logistic Regression model saved at: {model_path}")


Best parameters saved at: Output/data_full_random/best_params.pkl
Best Logistic Regression model saved at: Output/data_full_random/best_logistic_regression_model.pkl
Loaded Best Parameters: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 100, 'C': 0.46415888336127775}
Loaded Best Logistic Regression Model.


In [8]:
best_params_path = os.path.join(OUTPUT_DIR, 'best_params.pkl')
model_path = os.path.join(OUTPUT_DIR, 'best_logistic_regression_model.pkl')

In [9]:
# Best parameters 불러오기
best_params = joblib.load(best_params_path)
print(f"Loaded Best Parameters: {best_params}")

# Best model 불러오기
best_model = joblib.load(model_path)
print("Loaded Best Logistic Regression Model.")

Loaded Best Parameters: {'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 100, 'C': 0.46415888336127775}
Loaded Best Logistic Regression Model.


In [11]:
from sklearn.model_selection import GridSearchCV
# RandomizedSearchCV로 찾은 최적 범위를 기반으로 GridSearch 설정
param_grid = {
    'C': np.linspace(best_params['C'] - 1, best_params['C'] + 1, 50),  # 좁은 범위로 설정
    'solver': [best_params['solver']],  # 최적의 solver 선택
    'max_iter': [200, 300],  # 더 정밀하게
    'penalty': [best_params['penalty']]  # 최적 penalty 선택
}

grid_search = GridSearchCV(
    best_model,
    param_grid=param_grid,
    cv=5,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_tfidf, y_train)

print(f"Best Parameters from GridSearchCV: {grid_search.best_params_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits


: 

In [ ]:
model_f = LogisticRegression(**grid_search.best_params_)
model_f.fit(X_train_tfidf, y_train)

y_pred = model_f.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print(f"Hypertuning Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
OUTPUT_DIR_GRID = "Output/data_full_random/grid"

In [ ]:
# 저장할 디렉토리 설정 (필요 시 생성)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Best parameters 저장
best_params_path = os.path.join(OUTPUT_DIR, 'best_params.pkl')
joblib.dump(grid_search.best_params_, best_params_path)

print(f"Best parameters saved at: {best_params_path}")

# 모델 저장
model_path = os.path.join(OUTPUT_DIR, 'best_logistic_regression_model.pkl')
joblib.dump(model_f, model_path)

print(f"Best Logistic Regression model saved at: {model_path}")

In [11]:
import joblib
import os
# Best parameters 불러오기
OUTPUT_DIR_GRID = "Output/data_full_random"
best_params_path = os.path.join(OUTPUT_DIR_GRID, 'best_params.pkl')
model_path = os.path.join(OUTPUT_DIR_GRID, 'best_logistic_regression_model.pkl')

best_params = joblib.load(best_params_path)
print(f"Loaded Best Parameters: {best_params}")

# Best model 불러오기
best_model = joblib.load(model_path)
print("Loaded Best Logistic Regression Model.")

Loaded Best Parameters: {'C': 0.7702813323408695, 'max_iter': 300, 'penalty': 'l1', 'solver': 'liblinear'}
Loaded Best Logistic Regression Model.


In [13]:

from sklearn.metrics import accuracy_score, classification_report

In [12]:
# best_model = LogisticRegression(**best_params)
best_model.fit(X_train_tfidf, y_train)

y_pred = best_model.predict(X_test_tfidf)


NameError: name 'accuracy_score' is not defined

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Hypertuning Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Hypertuning Model Accuracy: 87.08%
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.86      0.87    136933
           2       0.87      0.88      0.87    143561

    accuracy                           0.87    280494
   macro avg       0.87      0.87      0.87    280494
weighted avg       0.87      0.87      0.87    280494

